<h1 style=\"text-align: center; font-size: 50px;\"> Image Super Resolution with FSRCNN Model MLflow Integration </h1>

# Notebook Overview
- Start Execution
- Install and Import Libraries
- Configure Settings

## Start Execution

In [ ]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [ ]:
start_time = time.time()  

logger.info("Notebook execution started.")

## Install and Import Libraries

In [ ]:
# ------------------------ Standard Library Imports ------------------------
from pathlib import Path
import argparse
import os
import copy
import warnings
import pandas as pd

# ------------------------ Third-Party Libraries ------------------------
import torch
from torch import nn
import base64
import io
from io import BytesIO
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# ------------------------ MLflow for Experiment Tracking and Model Management ------------------------
import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec

## Configure Settings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# ------------------------ MLflow Integration AND MODEL NAME------------------------
PROJECT_NAME = "super_resolution"
RUN_NAME = "fsrcnn_main"
MODEL_NAME = "fsrcnn"
MLFLOW_EXPERIMENT = "Super Resolution 4x"
# ------------------------- Paths -------------------------
TENSORLOGS_PATH = "/phoenix/tensorboard/tensorlogs"
TRAIN_PATH = "/home/jovyan/datafabric/DIV2K/DIV2K_train_HR/"
VALIDATION_PATH = "/home/jovyan/datafabric/DIV2K/DIV2K_valid_HR/"
BEST_MODEL_PATH = "best_model.pth"
MODEL_PATH = "FSRCNN_300_epochs.pt"

os.environ["TENSORBOARD_LOGDIR"] = "/phoenix/tensorboard"

## Verify Assets

In [ ]:
def log_asset_status(
    asset_path: str, asset_name: str, success_message: str, failure_message: str
) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")


log_asset_status(
    asset_path=TRAIN_PATH,
    asset_name="Train data",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio.",
)
log_asset_status(
    asset_path=VALIDATION_PATH,
    asset_name="Validation data",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio.",
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Model architecture and implementation

In [ ]:
class FSRCNN(nn.Module):
    """
    A class for building the architecture of the FSRCNN model.
    """

    def __init__(self, scale_factor):
        """
        Initializes the FSRCNN.
        Args:
            scale_factor: The factor by which to upscale the image.
        """
        try:
            super(FSRCNN, self).__init__()
            self.scale_factor = scale_factor

            self.feature_extraction = nn.Sequential(
                nn.Conv2d(3, 56, kernel_size=5, padding=2), nn.PReLU()
            )
            self.shrinking = nn.Sequential(nn.Conv2d(56, 12, kernel_size=1), nn.PReLU())
            self.non_linear_mapping = nn.Sequential(
                nn.Conv2d(12, 12, kernel_size=3, padding=1),
                nn.PReLU(),
                nn.Conv2d(12, 12, kernel_size=3, padding=1),
                nn.PReLU(),
                nn.Conv2d(12, 12, kernel_size=3, padding=1),
                nn.PReLU(),
                nn.Conv2d(12, 12, kernel_size=3, padding=1),
                nn.PReLU(),
                nn.Conv2d(12, 12, kernel_size=3, padding=1),
                nn.PReLU(),
            )
            self.expanding = nn.Sequential(nn.Conv2d(12, 56, kernel_size=1), nn.PReLU())
            self.deconvolution = nn.ConvTranspose2d(
                56,
                3,
                kernel_size=9,
                stride=scale_factor,
                padding=4,
                output_padding=scale_factor - 1,
            )

            logger.info("FSRCNN initialization done successfully")

        except Exception as e:
            logger.error(f"Error initializing FSRCNN: {str(e)}")
            raise

    def forward(self, x):
        """
        Implementation of the FSRCNN logic, in which, the input passes through every step of the arquiteture.
        Args:
            x: Low resolution image.

        Returns:
            x: High resolution image.
        """
        try:
            x = self.feature_extraction(x)
            x = self.shrinking(x)
            x = self.non_linear_mapping(x)
            x = self.expanding(x)
            x = self.deconvolution(x)
            return x
        except Exception as e:
            logger.error(f"Error implementing FSRCNN logic: {str(e)}")
            raise


## Logging Model to MLflow

In [ ]:
class FSRCNNModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Load model.
        """
        try:
            # Load the model
            model = FSRCNN(4)
            model.to(device)
            model_path = context.artifacts["fsrcnn_model"]
            model.load_state_dict(torch.load(model_path))
            model.eval()
            
            self.model = model


        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")
            raise

    def preprocess_image(self, base64_str):
        # Decode base64 string to image tensor
        img_data = base64.b64decode(base64_str)
        img = Image.open(BytesIO(img_data)).convert("RGB")
        img_np = np.array(img).astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0)
        return img_tensor

    def postprocess_image(self, output_tensor):
        # Convert output tensor to base64 string
        output_np = output_tensor.squeeze().permute(1, 2, 0).cpu().numpy()
        output_img = (output_np * 255).clip(0, 255).astype(np.uint8)
        output_pil = Image.fromarray(output_img)
        buffer = BytesIO()
        output_pil.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode("utf-8")

    def predict(self, context, model_input):
    
        try:
            results = []
            for base64_str in model_input["image"]:
                input_tensor = self.preprocess_image(base64_str).to(device)
                with torch.no_grad():
                    output_tensor = self.model(input_tensor)
                output_base64 = self.postprocess_image(output_tensor)
                results.append(output_base64)
            return results

        except Exception as e:
            logger.error(f"Error performing prediction: {str(e)}")
            raise
    
    @classmethod
    def log_model(cls, model_name):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        try:
            # Define input and output schema
            
            input_schema = Schema([ColSpec("string", "image")])

            output_schema = Schema([ColSpec("string", name = "prediction")])

            # Define model signature
            signature = ModelSignature(inputs=input_schema, outputs=output_schema)
            
            # Log the model in MLflow
            mlflow.pyfunc.log_model(
                model_name,
                python_model=cls(),
                artifacts={
                    "fsrcnn_model": MODEL_PATH,           
                },
                signature=signature,
                pip_requirements = [
                "torch",
            ]

            )
        except Exception as e:
            logger.error(f"Error logging model: {str(e)}")
            raise   

In [ ]:
logger.info(f"Starting the experiment: {MLFLOW_EXPERIMENT}")

# Set the MLflow experiment name
mlflow.set_tracking_uri("/phoenix/mlflow")
mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT)

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME) as run:
    # Print the artifact URI for reference
    logging.info(f"Run's Artifact URI: {run.info.artifact_uri}")

    # Log the model to MLflow
    FSRCNNModel.log_model(model_name=MODEL_NAME)

    # Register the logged model in MLflow Model Registry
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/{MODEL_NAME}", name=MODEL_NAME
    )

logger.info(f"Registered the model: {MODEL_NAME}")

## Fetching the Latest Model Version from MLflow

In [ ]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the model
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version  # Extract the latest model version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(
    f"models:/{MODEL_NAME}/{latest_model_version}"
)

# Print the latest model version and its signature
logger.info(f"Latest Model Version: {latest_model_version}")
logger.info(f"Model Signature: {model_info.signature}")

## Loading the Model and Running Inference

In [ ]:
# Load the model
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{MODEL_NAME}/{latest_model_version}"
)

# Your base64 image
image_base64 = "..."

# Decode the input image
image_bytes = base64.b64decode(image_base64)
image_input = Image.open(BytesIO(image_bytes))

# Create a DataFrame with the 'image' column
input_df = pd.DataFrame({"image": [image_base64]})

# Perform inference
output = model.predict(input_df)

# Decode the output image
output_image_base64 = output[0]
output_image_bytes = base64.b64decode(output_image_base64)
output_image = Image.open(BytesIO(output_image_bytes))

# Display input and output images side by side
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image_input)
axes[0].set_title("Input Image")
axes[0].axis("off")

axes[1].imshow(output_image)
axes[1].set_title("Output Image")
axes[1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")